d
## ENSF 612 - Assignment 3
### Author: Steven Duong (30022492)

In [0]:
%pip install nltk
%pip install pyspellchecker
%pip install beautifulsoup4

Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.


### Question 1
### Levenshtein Distance Algorithm

In [0]:
from spellchecker import SpellChecker
import nltk
nltk.download('punkt')

# Initialize the spell checker
spell = SpellChecker()

# The sentence with typos to be fixed.
sentence = "this is asignmnt 2 and it is jsut great!"
fixed_sentence = ""

# Loop through each word in the sentence and find the correct word using spellchecker.
for word in nltk.word_tokenize(sentence):
    # Check if the word is misspelled and correct it if it is.
    corrected_word = spell.correction(word)
    # Add the corrected word to the fixed sentence.
    fixed_sentence += corrected_word + " "

print(fixed_sentence.replace('2', '3'))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
this is assignment 3 and it is just great ! 


Here there are two typos in the sentence:
1. just
- Replace the 's' with a 'u' and the 'u' with an 's'. (2 replacements).

2. assignment
- Insert characters 's' and and 'e' to fix the word typo. (2 insertions).

The number 2 was replaced with a 3 without the need for the algorithm.

### Question 2
### Stack Overflow Similarity Analysis

In [0]:
# Define a function to load a CSV file into a PySpark DataFrame
def load(file):
    # Use the read method of the SparkSession to read the CSV file
    df = (spark.read
        .option("multiline", "true")  # Enable reading multiline data
        .option("quote", '"')         # Set the quote character
        .option("header", "true")     # Treat the first row as the header row
        .option("escape", "\\")       # Set the escape character for backslashes
        .option("escape", '"')        # Set the escape character for quotes
        .csv(filepath)
        )
    
    return df

# Load the three CSV files using the load function
df_spark = load_csv('/FileStore/tables/SO_Spark.csv')
df_ml = load_csv('/FileStore/tables/SO_ML.csv')
df_security = load_csv('/FileStore/tables/SO_Security.csv')

# Cast the Score column to integer type for each DataFrame
df_spark = df_spark.withColumn("Score", df_spark["Score"].cast("int"))
df_ml = df_ml.withColumn("Score", df_ml["Score"].cast("int"))
df_security = df_security.withColumn("Score", df_security["Score"].cast("int"))


In [0]:
# Define a UDF to merge two columns
@udf
def merge_cols(a, b):
    """
    UDF to merge two columns
    """
    c = a + " " + b  
    return c

# Apply the UDF to create a new column "TitleBody" that merges "Title" and "Body" columns, and drop "Body" column
df_spark = df_spark.select("*", merge_cols("Title", "Body").alias("Title_and_Body")).drop("Body")
df_ml = df_ml.select("*", merge_cols("Title", "Body").alias("Title_and_Body")).drop("Body")
df_security = df_security.select("*", merge_cols("Title", "Body").alias("Title_and_Body")).drop("Body")

#### 1. Preprocessing of Text

In [0]:
# Import the BeautifulSoup library to parse HTML
from bs4 import BeautifulSoup

# Define a UDF (user-defined function) to preprocess text data
@udf
def preprocess_text(text):

    # Parse the input text as HTML using BeautifulSoup
    soup = BeautifulSoup(text)

    # Find all hyperlinks in the HTML and remove them if present
    urls = soup.find_all('a')
    if len(urls) > 0:
        soup.a.clear()

    # Find all code blocks in the HTML and remove them if present
    codes = soup.find_all('code')
    if len(codes) > 0:
        soup.code.clear()

    # Find all pre-formatted text blocks in the HTML and remove them if present
    pres = soup.find_all('pre')
    if len(pres) > 0:
        soup.pre.clear()

    # Extract the remaining text from the cleaned HTML and convert it to lowercase
    text = soup.get_text().lower()

    # Return the preprocessed text
    return text

# Apply the preprocess_text UDF to the "Title_and_Body" column of each DataFrame,
# creating a new column called "Text", and drop the original "Title_and_Body" column
df_spark = df_spark.select("*", preprocess_text("Title_and_Body").alias("Text")).drop("Title_and_Body")
df_ml = df_ml.select("*", preprocess_text("Title_and_Body").alias("Text")).drop("Title_and_Body")
df_security = df_security.select("*", preprocess_text("Title_and_Body").alias("Text")).drop("Title_and_Body")

##### a) Tokenization and removing stopwords

In [0]:
# Import the necessary NLTK modules
import nltk
nltk.download('stopwords')
nltk.download('punkt')

# Define a UDF (user-defined function) to remove stopwords from text data
@udf
def remove_all_stopwords(text):
  
    # Load the list of English stopwords from NLTK
    stopwords = nltk.corpus.stopwords.words('english')

    # Tokenize the input text into sentences using NLTK's sentence tokenizer
    sentences = nltk.sent_tokenize(text)

    # Initialize an empty list to store the filtered words
    filt_words = []

    # Loop over each sentence in the tokenized text
    for sentence in sentences:
        # Tokenize the sentence into words using NLTK's word tokenizer
        words = nltk.word_tokenize(sentence)

        # Loop over each word in the tokenized sentence
        for word in words:
            # Skip any words that are less than 3 characters long
            if len(word) < 3:
                continue

            # Skip any words that are in the list of stopwords
            if word in stopwords:
                continue

            # Add the word to the list of filtered words
            filt_words.append(word)

    # Return the filtered words as a list
    return filt_words

# Apply the remove_all_stopwords UDF to create a new column "Filtered_Text" that removes all stopwords in "Text" column, and drop "Text" column
df_spark = df_spark.select("*", remove_all_stopwords("Text").alias("Filtered_Text")).drop("Text")
df_ml = df_ml.select("*", remove_all_stopwords("Text").alias("Filtered_Text")).drop("Text")
df_security = df_security.select("*", remove_all_stopwords("Text").alias("Filtered_Text")).drop("Text")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


##### b) Removing noise

In [0]:
# Import the regular expression module
import re

# Define a regular expression pattern to match punctuation characters
punct_regex = re.compile('^[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]+$')

# Define a UDF (user-defined function) called remove_punct to remove punctuation from text data
@udf
def remove_punct(text):
    
    # Initialize an empty list to store the cleaned text
    cleaned_text = []

    # Loop through each word in the input text
    for word in text:
        # Use the regular expression pattern to check if the word contains only punctuation characters
        if bool(punct_regex.match(word)) == False:
            # If the word does not contain only punctuation, add it to the cleaned text list
            cleaned_text.append(word)

    # Return the cleaned text list as the output of the UDF
    return cleaned_text

# Apply the remove_punct UDF to create a new column "Text_no_punct" that removes punctuation from "Filtered_Text" column, and drop "Filtered_Text" column
df_spark = df_spark.select("*", remove_punct("Filtered_Text").alias("Text_no_punct")).drop("Filtered_Text")
df_ml = df_ml.select("*", remove_punct("Filtered_Text").alias("Text_no_punct")).drop("Filtered_Text")
df_security = df_security.select("*", remove_punct("Filtered_Text").alias("Text_no_punct")).drop("Filtered_Text")


##### c) Stemming the text

In [0]:
# Import the Natural Language Toolkit (nltk) module
import nltk

# Define a UDF (user-defined function) called stem_words to stem words in text data
@udf
def stem_words(text):

    # Create a Snowball stemmer for the English language
    stemmer = nltk.stem.SnowballStemmer(language='english')

    # Initialize an empty list to store the stemmed words
    stemmed_text = []

    # Loop over each word in the input text
    for word in text:
        # Use the Snowball stemmer to stem the word
        stemmed_word = stemmer.stem(word)

        # Add the stemmed word to the list of stemmed words
        stemmed_text.append(stemmed_word)

    # Return the list of stemmed words as the output of the UDF
    return stemmed_text

# Apply the stem_words UDF to create a new column "Stemmed_Text" that stems words in "Text_Filtered" column, and drop "Text_Filtered" column
df_spark = df_spark.select("*", stem_words("Text_no_punct").alias("Stemmed_Text")).drop("Text_no_punct")
df_ml = df_ml.select("*", stem_words("Text_no_punct").alias("Stemmed_Text")).drop("Text_no_punct")
df_security = df_security.select("*", stem_words("Text_no_punct").alias("Stemmed_Text")).drop("Text_no_punct")


#### Highest scored question with no accepted answer

In [0]:
# Returns the highest-scoring unanswered question from a PySpark DataFrame of Stack Overflow questions.
def highest_unanswered_questions(df):
    
    # Filter the DataFrame to keep only unanswered questions
    unanswered_df = df.filter(df["AcceptedAnswerId"].isNull())
    
    # Sort the unanswered questions by score in descending order
    unanswered_df = unanswered_df.sort("Score", ascending=False)
    
    # Return the first row of the sorted DataFrame (i.e. the highest-scoring unanswered question)
    return unanswered_df.first()

# Call the highest_unanswered_questions function on each of the input PySpark DataFrames and store the resulting Rows in variables
qs_spark = highest_unanswered_questions(df_spark)
qs_ml = highest_unanswered_questions(df_ml)
qs_security = highest_unanswered_questions(df_security)


#### Defining the cosine similarity function

In [0]:
import math
import nltk

# Define a UDF (user-defined function) to calculate the cosine similarity of top unanswered question (text2) to answered questions (text1)
@udf
def cosine_similarity(text1):
    
    # Combine the unique words from both texts
    d = list(set(text1 + text2))

    # Initialize arrays to hold the presence/absence of words in each text
    dt1 = []
    dt2 = []

    # Loop over each word in the combined set of words
    for word in d:
        # Check if the word is present in text t1
        if word in text1:
            dt1.append(1)
        else:
            dt1.append(0)

        # Check if the word is present in text t2
        if word in text2:
            dt2.append(1)
        else:
            dt2.append(0)

    # Calculate the numerator and denominator of the cosine similarity formula
    numer = 0
    sum_sq_dt1 = 0
    sum_sq_dt2 = 0

    for i in range(0, len(dt1)):
        numer += dt1[i] * dt2[i]
        sum_sq_dt1 += dt1[i] * dt1[i]
        sum_sq_dt2 += dt2[i] * dt2[i]

    denom = math.sqrt(sum_sq_dt1 * sum_sq_dt2)

    # Calculate the cosine similarity score
    if denom == 0:
        return 0

    return numer/denom


# Define a function to return a PySpark DataFrame with a similarity column calculated using cosine similarity
def df_with_similarity(df):

    # Filter the DataFrame to include only questions with an accepted answer, and calculate the cosine similarity score for each row
    df = df.filter(df["AcceptedAnswerId"].isNotNull()).select("*", cosine_similarity("Stemmed_text").alias("Similarity")).sort("Similarity", ascending=False)

    # Return the DataFrame with the similarity column
    return df


#### Calculating similarity scores for each question in all 3 files

In [0]:
# Apply the df_with_similarity() function to each of the three DataFrames from earlier

text2 = qs_spark["Stemmed_Text"].strip('][').split(', ')
df_spark_with_answers = df_with_similarity(df_spark)

text2 = qs_ml["Stemmed_Text"].strip('][').split(', ')
df_ml_with_answers = df_with_similarity(df_ml)

text2 = qs_security["Stemmed_Text"].strip('][').split(', ')
df_security_with_answers = df_with_similarity(df_security)


#### Printing the recommended questions for the top unanswered question in each file

In [0]:
def print_recommendations(df, n):

    # Get the first n row from the DF
    num_top_questions = df.take(n)

    # Loop over each row in the top n rows and print the Title, Id, and Similarity Score as a percentage with 2 decimal places
    for i in range(n):
        similarity_score = round(float(num_top_questions[i]["Similarity"]) * 100, 2)
        print("  {}. [Id = {}, similarity score = {}%] {}".format(i+1, num_top_questions[i]["Id"], similarity_score, num_top_questions[i]["Title"]))


In [0]:
# Define a list of tuples, where each tuple contains a DataFrame and a string representing the name of the DataFrame
dataframes = [(df_spark, "SO_Spark"), (df_ml, "SO_ML"), (df_security, "SO_Security")]
dataframes_ans = [df_spark_with_answers, df_ml_with_answers, df_security_with_answers]
i = 0

# Loop over each tuple in the list
for df_tuple in dataframes:
    # Get the highest scored question in the DataFrame
    q = highest_unanswered_questions(df_tuple[0])
    # Print the highest scored question
    print("The highest scored question in {} without any accepted answer is:\n  [Id = {}] {}\n".format(df_tuple[1], q["Id"], q["Title"]))
    
    # Print the top three recommendations with accepted answers using the print_recommendations function
    print("Here are the top three recommendations with accepted answers:")
    print_recommendations(dataframes_ans[i], 3)
    i+=1
    print("\n")


The highest scored question in SO_Spark without any accepted answer is:
  [Id = 21138751] Spark java.lang.OutOfMemoryError: Java heap space

Here are the top three recommendations with accepted answers:
  1. [Id = 38223723, similarity score = 23.78%] Output file is getting generated on slave machine in apache spark
  2. [Id = 44094070, similarity score = 23.57%] Why does standalone master schedule drivers on a worker?
  3. [Id = 34324106, similarity score = 22.22%] Spark cluster: Standalone mode without HDFS


The highest scored question in SO_ML without any accepted answer is:
  [Id = 4752626] Epoch vs Iteration when training neural networks

Here are the top three recommendations with accepted answers:
  1. [Id = 27959831, similarity score = 19.92%] How does Spark read 100K images effeciently?
  2. [Id = 38536468, similarity score = 19.1%] Got OutOfMemory when run Spark MLlib kmeans
  3. [Id = 54759966, similarity score = 17.77%] How to pass multiple Columns as features in a Logistic